# This notebook will aim to use the trained models weights and load it on the inference model to generate predictions and novel sequences

In [44]:
from models.arcs import generate, GenPhiloTextA
from utilities.loaders import load_file
from utilities.preprocessors import preprocess, get_chars, map_value_to_index, decode_predictions
import tensorflow as tf

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load raw data

In [45]:
corpus = load_file('./data/notes.txt')
corpus = preprocess(corpus)
chars = get_chars(corpus)
char_to_idx = map_value_to_index(chars)
idx_to_char = map_value_to_index(chars, inverted=True)

In [46]:
char_to_idx.get_vocabulary()

['[UNK]',
 '\n',
 ' ',
 '!',
 '"',
 '&',
 "'",
 '(',
 ')',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '´',
 'ç',
 'é',
 'ï',
 '—',
 '…']

In [47]:
n_unique = len(char_to_idx.get_vocabulary())

# Declare same hyper params used in training model

In [48]:
emb_dim = 256
n_a = 512
T_x = 100
dense_layers_dims = [n_unique]
batch_size = 128
alpha = 1e-3
lambda_ = 0.8
drop_prob = 0.4
normalize = False
n_epochs =  100

# Declare sample input to build inference model and access .summary()

In [49]:
sample_input = tf.random.uniform(shape=(1, T_x), minval=0, maxval=n_unique - 1, dtype=tf.int32)
sample_h = tf.zeros(shape=(1, n_a))
sample_c = tf.zeros(shape=(1, n_a))

# Redeclare architecture by passing the same hyper params used in training and then load weights

In [50]:
saved_model = GenPhiloTextA(emb_dim=emb_dim, n_a=n_a, n_unique=n_unique, dense_layers_dims=dense_layers_dims, drop_prob=drop_prob, normalize=normalize)
saved_model(sample_input)

<tf.Tensor: shape=(1, 100, 57), dtype=float32, numpy=
array([[[-0.00053872,  0.0049823 ,  0.00049064, ..., -0.00352435,
          0.00734192, -0.00313014],
        [ 0.0037324 ,  0.00581681,  0.00823732, ..., -0.00497193,
          0.00739325, -0.000217  ],
        [ 0.00275159,  0.00910855,  0.00258863, ..., -0.00403328,
         -0.00098546,  0.007524  ],
        ...,
        [ 0.00238691,  0.00959118, -0.00426605, ..., -0.00525967,
         -0.01164197,  0.00737282],
        [-0.0030315 ,  0.00058106,  0.00561463, ..., -0.00566912,
         -0.00125402,  0.00508967],
        [-0.01124973,  0.004311  ,  0.00307721, ..., -0.00260225,
          0.00539622,  0.00212805]]], dtype=float32)>

In [51]:
sample_input = tf.random.uniform(shape=(1, T_x + 10), minval=0, maxval=n_unique - 1, dtype=tf.int32)
saved_model(sample_input)

<tf.Tensor: shape=(1, 110, 57), dtype=float32, numpy=
array([[[ 3.2126354e-03, -5.9258621e-03, -2.4557475e-03, ...,
          1.0246720e-02,  9.4786370e-03, -7.8828947e-05],
        [ 5.6936350e-03, -2.7485944e-03, -5.0903130e-03, ...,
          1.4657163e-02, -1.7278363e-03, -2.0295780e-03],
        [ 7.9504717e-03, -7.0794951e-03, -8.1595767e-04, ...,
          1.1122698e-02,  1.3763035e-03, -2.4517188e-03],
        ...,
        [-1.0152766e-03, -9.1297086e-03,  1.1756338e-02, ...,
          4.7399439e-03, -1.0114578e-03, -5.7670977e-03],
        [-1.8755067e-04, -8.7925531e-03,  1.1674371e-02, ...,
          1.2707835e-02,  4.5867646e-03, -1.0848419e-02],
        [-3.7435018e-03, -6.6359658e-03,  8.6071240e-03, ...,
          1.0488454e-02,  9.7658881e-04, -2.7265165e-03]]], dtype=float32)>

In [52]:
saved_model.trainable_weights

[<tf.Variable 'gen_philo_text_a_2/character-lookup/embeddings:0' shape=(57, 256) dtype=float32, numpy=
 array([[-0.02532429, -0.02236536, -0.0374039 , ...,  0.03197709,
          0.04618976, -0.00566719],
        [ 0.01729969,  0.01035541,  0.00585414, ...,  0.04315854,
          0.0494112 , -0.0113189 ],
        [ 0.03775368, -0.02040966, -0.02018896, ..., -0.04924528,
         -0.00782677, -0.04266036],
        ...,
        [-0.00483074,  0.01806113,  0.02739585, ..., -0.03169592,
          0.01905766, -0.04246932],
        [-0.02227191, -0.01520914, -0.00665072, ...,  0.01178259,
         -0.02600683, -0.03053491],
        [-0.01939147, -0.02655815,  0.02445975, ...,  0.01116021,
         -0.04833468,  0.04599574]], dtype=float32)>,
 <tf.Variable 'gen_philo_text_a_2/lstm-layer/lstm_cell/kernel:0' shape=(256, 2048) dtype=float32, numpy=
 array([[-0.02837389,  0.03977501, -0.00153487, ..., -0.01582713,
          0.01428244, -0.0017515 ],
        [-0.00109827, -0.02651026,  0.02153903,

In [53]:
saved_model.summary()

Model: "gen_philo_text_a_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 character-lookup (Embeddin  multiple                  14592     
 g)                                                              
                                                                 
 lstm-layer (LSTM)           multiple                  1574912   
                                                                 
 dense-layer-0 (Dense)       multiple                  29241     
                                                                 
Total params: 1618745 (6.18 MB)
Trainable params: 1618745 (6.18 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [54]:
saved_model.load_weights(filepath='./saved/weights/notes_gen_philo_text_a_100_3.0299.h5')

In [55]:
saved_model.trainable_weights

[<tf.Variable 'gen_philo_text_a_2/character-lookup/embeddings:0' shape=(57, 256) dtype=float32, numpy=
 array([[-2.27964771e-31, -1.04065716e-31,  6.76143058e-32, ...,
          5.78990514e-33, -6.32452695e-35,  1.94203334e-32],
        [-2.33196533e-06, -5.57359442e-07,  7.22897767e-07, ...,
          3.29896963e-08,  4.24313072e-07, -7.51945777e-08],
        [-6.22333800e-06, -8.55494363e-06,  1.20172226e-05, ...,
          3.50521004e-05,  2.60806992e-05, -1.78583414e-05],
        ...,
        [ 1.16941873e-11,  5.07066784e-11,  2.42132052e-12, ...,
          1.95506927e-11, -9.94128564e-12,  5.03835099e-11],
        [-1.29197097e-09, -5.15809404e-08,  6.50408394e-09, ...,
          1.08325215e-08,  1.27126455e-07, -2.77360197e-08],
        [ 6.44838792e-07, -1.91505521e-07,  9.48950856e-08, ...,
          1.13875785e-06,  8.59064301e-07, -3.50037581e-08]], dtype=float32)>,
 <tf.Variable 'gen_philo_text_a_2/lstm-layer/lstm_cell/kernel:0' shape=(256, 2048) dtype=float32, numpy=
 arra

# generate characters using prompt

In [56]:
prompts = ["hello"]
pred_ids_a = generate(saved_model, prompts=prompts, char_to_idx=char_to_idx, temperature=1.0)
pred_ids_b = generate(saved_model, prompts=prompts, char_to_idx=char_to_idx, temperature=0.8)
pred_ids_c = generate(saved_model, prompts=prompts, char_to_idx=char_to_idx, temperature=0.6)
pred_ids_d = generate(saved_model, prompts=prompts, char_to_idx=char_to_idx, temperature=0.4)

tf.Tensor([[32 29 36 36 39]], shape=(1, 5), dtype=int64)
tf.Tensor(
[[[-9.7952789e-01 -4.9550407e-02  7.8760695e-01 -9.4414473e-01
   -9.1001999e-01 -9.8696762e-01 -1.0040931e+00 -9.8399913e-01
   -9.6389389e-01 -1.4195003e-03 -9.9328291e-01 -3.3886015e-01
   -9.5995611e-01 -9.6912700e-01 -9.7229034e-01 -1.0030506e+00
   -9.4810855e-01 -1.0059516e+00 -9.9453384e-01 -9.9514633e-01
   -9.7885209e-01 -1.0189408e+00 -9.2216438e-01 -9.6369439e-01
   -9.5581770e-01  9.0898013e-01  3.4355694e-01  6.2924385e-01
    8.2376647e-01  9.0805137e-01  7.2464496e-01  5.3681183e-01
    9.0523934e-01  8.8704902e-01 -9.1242909e-01 -4.0618968e-01
    8.0495793e-01  6.1087257e-01  8.9836323e-01  9.1019720e-01
    2.1426669e-01 -9.6541142e-01  8.5857725e-01  9.0976363e-01
    9.2935693e-01  7.0334429e-01  2.4025665e-01  5.6967038e-01
   -9.3542272e-01  5.4357767e-01 -9.6410286e-01 -9.4257683e-01
   -9.8725337e-01 -9.6333629e-01 -1.0216787e+00 -9.2859411e-01
   -9.4580013e-01]
  [-1.1025574e+00 -2.9315751e-0

In [57]:
pred_ids_a

<tf.Tensor: shape=(1, 255), dtype=int64, numpy=
array([[32, 29, 36, 36, 39, 49, 25, 32, 38,  2, 29,  3, 49, 26, 39, 44,
        44, 25,  2, 33, 43, 43, 44, 25,  2, 32, 33,  1, 31, 36, 29, 32,
        42, 33, 33,  2,  2, 29,  2, 46,  2,  2,  2, 25, 32, 25, 38, 47,
         2, 29, 32,  2, 25, 32, 29,  2, 26, 29,  2, 32, 44,  2,  2, 27,
        29, 43, 30, 39, 29,  2, 36, 22, 37, 39, 29, 33, 39, 29, 44, 39,
         6, 32, 45,  2, 29, 32, 39, 55,  2, 43, 29, 55,  2, 29, 27, 28,
        39,  2, 45, 44, 22, 29, 33, 25, 39, 25, 38, 32, 40,  2, 45, 41,
        45,  2, 35, 32, 37, 25,  2, 25, 47, 39, 29, 29, 19, 25,  2,  2,
        42, 32, 38, 39, 32, 38, 29,  2, 49, 39,  2, 36,  2, 29, 11, 28,
        20, 39,  2, 33,  2, 29, 39, 44, 37, 36, 30, 42, 44,  2,  9,  2,
        43, 29, 27, 32, 16,  2, 29, 43,  2, 38,  2, 43, 27, 29,  2, 47,
        16, 29, 39, 43, 43, 40, 30,  2, 49, 44,  2, 29,  2,  2, 33, 29,
        33, 29,  3, 31, 43,  2, 32, 39, 29, 44,  2, 55,  2, 33,  2,  2,
        43, 28, 

# Extract trainable layers

In [58]:
decode_predictions(pred_ids_a, idx_to_char)

"helloyahn e!ybotta issta hi\nglehrii  e v   ahanw eh ahe be ht  cesfoe l:moeioeto'hu eho— se— ecdo ut:eiaoanhp uqu khma awoee7a  rhnohne yo l e.d8o i eotmlfrt , sech3 es n sce w3eosspf yt e  ieie!gs hoet — i  sdnasah mc8n 6  d\nw  e dvg2eoeo mnn s meée orrh"

In [59]:
decode_predictions(pred_ids_b, idx_to_char)

'hellob ealoo ijai orarsfntehahv  nf r rhnluhnic e enati w ndbi  gheeotne a n onaeiitmtttaee" b alt   vienoal foei n iahlo r ewrdlaniidonsn  ithepre t a  aeeh ehti cdnotilf6i t sangaaisgaa i   ihssl aiir harea ss te hyh eogo  f aeimhfrsocfe ooev   lnl,ei n'

In [60]:
decode_predictions(pred_ids_c, idx_to_char)

'hellow i ouri h  ottiae aite le  a  ts o aioeeo,  h udt t   ia ywiee  s a  de    pi e   tae nee tit  t slet   at    tt oeg  oe   f ttit nt  eirlt ey a ena   ryi t y d  n etlc ene u nitoiieh nl isttni o reioc t eectnanghot a   ´aeat btst eaeetri c ttolo sg'

In [61]:
decode_predictions(pred_ids_d, idx_to_char)

"hello 'a   i  ie  s t        h  e  eet e     i    eon   ie     n    o    e        b o   ee      i    to r     nl it e   o e   r ei e e eo   t ea    e o  t eneca        est sa        a     tiy e  ee   e  i   se   ph  e    tseen ao ai e  w  e    o w  o     "